In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
import random

# Function to compute propensity scores
def compute_propensity_scores(df, relevant_columns):
    X = df[relevant_columns]
    model = LogisticRegression()
    model.fit(X, df['group'])
    scores = model.predict_proba(X)[:, 1]
    return scores

def propensity_score_matching(control, treatment, caliper):
    matched_treatment = pd.DataFrame()
    matched_treatment_indices = set()

    for index, control_row in control.iterrows():
        control_score = control_row['propensity_score']
        potential_matches = treatment[(treatment['propensity_score'] >= control_score - caliper) &
                                      (treatment['propensity_score'] <= control_score + caliper) &
                                      (~treatment.index.isin(matched_treatment_indices))
                                      ]
        if not potential_matches.empty:
            # Select up to 5 matches without replacement
            selected_matches = potential_matches.sample(n=min(5, len(potential_matches)), replace=False)
            matched_treatment = pd.concat([matched_treatment, selected_matches])
            matched_treatment_indices.update(selected_matches.index)

            # Add matched treatment indices to the set to avoid re-matching
            matched_treatment_indices.update(selected_matches.index)

    return matched_treatment

# Load the datasets
control_group_df = pd.read_csv(fr"E:\Propensity score matching analysis\TAZ_destination.csv")
treatment_group_df = pd.read_csv(fr"E:\STP_PWoD\Data\TAZ_destination_PwoD.csv")

# # Remove null or NA values from CO_NAME column
# control_group_df.dropna(subset=['CO_NAME'], inplace=True)
# treatment_group_df.dropna(subset=['CO_NAME'], inplace=True)

# Unique values for emp_text column in control_group_df
control_emp_values = control_group_df['emp_text'].unique()
# Filtering treatment_group_df to have only those values in control_emp_values
treatment_group_df = treatment_group_df[treatment_group_df['emp_text'].isin(control_emp_values)]

# # Unique values for CO_NAME column in control_group_df
# control_CO_values = control_group_df['CO_NAME'].unique()
# # Filtering treatment_group_df to have only those values in control_emp_values
# treatment_group_df = treatment_group_df[treatment_group_df['CO_NAME'].isin(control_CO_values)]

# Keep only unique values in the 'hhmemberid' column
control_group_df = control_group_df.drop_duplicates(subset=['hhmemberid'])
treatment_group_df = treatment_group_df.drop_duplicates(subset=['hhmemberid'])

# define categorical variables
cat=['age', 'gender', 'education', 'licensed', 'emp_text']
# define continuous variables
con=['num_veh', 'workers', 'hhsize']

# Define relevant columns for the analysis
relevant_columns = cat+con

# Encode categorical variables
le = LabelEncoder()
for col in cat:
    if control_group_df[col].dtype == object:
        control_group_df[col] = le.fit_transform(control_group_df[col])
        treatment_group_df[col] = le.transform(treatment_group_df[col])

# Assign group labels
control_group_df = control_group_df.assign(group=0)
treatment_group_df = treatment_group_df.assign(group=1)

# Combine subsets for propensity score computation
combined_subset = pd.concat([control_group_df, treatment_group_df])
combined_subset['propensity_score'] = compute_propensity_scores(combined_subset, relevant_columns)

# Split the combined subset back into control and treatment with propensity scores
control_subset = combined_subset[combined_subset['group'] == 0]
treatment_subset = combined_subset[combined_subset['group'] == 1]

# Define caliper as 0.25 standard deviations of the propensity score
caliper = 0.25 * combined_subset['propensity_score'].std()

matched_subset = propensity_score_matching(control_subset, treatment_subset, caliper)

display(matched_subset)

,hptripid,tripID,hhmemberid,o_purp_t,depart_hhm,mode_t,hhsize,workers,hh_income,num_veh,...,d_CoTAZID_v30,ACRES,DEVACRES,DEVPBLEPCT,X,Y,CO_NAME,CITY_NAME,group,propensity_score
11214,W19968FT.01.01,1,W19968FT.01,home,1115,transit,3,1,5,1,...,350146,390.484860,390.484860,1.0,428763.736864,4.512850e+06,SALT LAKE,Salt Lake City,1,0.996454
27037,W74613KS.02.01,1,W74613KS.02,home,710,auto,2,2,4,2,...,350270,180.408777,180.408777,1.0,419476.803211,4.510876e+06,SALT LAKE,Salt Lake City,1,0.994689
13134,W26387KR.02.01,1,W26387KR.02,home,1225,auto,4,2,5,2,...,350968,82.245493,82.245493,1.0,424807.385071,4.493006e+06,SALT LAKE,Sandy,1,0.992095
16875,W38972TF.02.01,1,W38972TF.02,home,800,auto,4,1,5,2,...,350222,14.377894,14.377894,1.0,424181.433779,4.512455e+06,SALT LAKE,Salt Lake City,1,0.996413
17908,W42565NK.01.01,1,W42565NK.01,home,745,auto,2,2,6,2,...,350848,194.563923,194.563923,1.0,420190.513288,4.494613e+06,SALT LAKE,West Jordan,1,0.996224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4341,M23430FR.01.01,1,M23430FR.01,work,600,auto,5,1,5,2,...,490148,212.576394,212.576394,1.0,415021.469824,4.463269e+06,UTAH,Eagle Mountain,1,0.996548
17518,W41422RE.02.01,1,W41422RE.02,home,1130,auto,5,1,5,2,...,110028,322.965080,322.965080,1.0,409859.047353,4.555243e+06,DAVIS,Clinton,1,0.995335
15639,W34992DZ.01.01,1,W34992DZ.01,home,805,walk,2,2,6,2,...,570279,165.725154,165.725154,1.0,420098.405715,4.561620e+06,WEBER,Ogden,1,0.993989
26587,W72954CS.01.01,1,W72954CS.01,home,725,auto,2,2,5,1,...,350060,2040.917532,2040.917532,1.0,417426.070458,4.514919e+06,SALT LAKE,Salt Lake City,1,0.991545


In [2]:
# Calculate the standard deviation of the propensity scores in the combined matched dataset
propensity_score_std = matched_subset['propensity_score'].std()
print("Standard Deviation of Propensity Scores in the Matched Dataset:", propensity_score_std)

Standard Deviation of Propensity Scores in the Matched Dataset: 0.11179289514149957


In [3]:
### Keep all matching hhmemberid from the initial treatment dataset
# Load the initial_treatment dataset
initial_treatment = pd.read_csv(fr"E:\STP_PWoD\Data\TAZ_destination_PwoD.csv")
# Merge the two dataframes on hhmemberid
matching_rows = pd.merge(initial_treatment, matched_subset[['hhmemberid']], on='hhmemberid', how='inner')
print('number of trips after matching the person data is:',len(matching_rows))
# Save the combined matched dataset to a CSV file
matching_rows.to_csv(fr"E:\STP_PWoD\Data\TAZ_destination_PwoD_PSM1.csv", index=False)

number of trips after matching the person data is: 1916


In [4]:
#### Keeping only matching hptripids in origin and destination dataset
# Load the datasets
TAZ_origin_df = pd.read_csv(fr"E:\STP_PWoD\Data\TAZ_origin_PwoD_PSM1.csv")
TAZ_destination_df = pd.read_csv(fr"E:\STP_PWoD\Data\TAZ_destination_PwoD_PSM1.csv")

# Finding unique hptripid in each file
unique_origin = set(TAZ_origin_df['hptripid'].unique())
print("Number of unique hptripid in origin:", len(unique_origin))
unique_destination = set(TAZ_destination_df['hptripid'].unique())
print("Number of unique hptripid in destination:", len(unique_destination))

# Common hptripid in both origin and destination files
common_hptripid = unique_origin.intersection(unique_destination)
num_common_hptripid = len(common_hptripid)

# Keeping only common hptripid in both files
TAZ_origin_common = TAZ_origin_df[TAZ_origin_df['hptripid'].isin(common_hptripid)]
TAZ_destination_common = TAZ_destination_df[TAZ_destination_df['hptripid'].isin(common_hptripid)]
# Saving the matching hptripd origin and destination dataset data as a CSV file
TAZ_origin_common.to_csv(fr"E:\STP_PWoD\Data\TAZ_origin_PwoD_PSM2.csv", index=False)
TAZ_destination_common.to_csv(fr"E:\STP_PWoD\Data\TAZ_destination_PwoD_PSM2.csv", index=False)
# Print the results
print("Number of common hptripid in both files:", num_common_hptripid)

Number of unique hptripid in origin: 1880
Number of unique hptripid in destination: 1916
Number of common hptripid in both files: 484
